In [2]:
import pandas as pd
df = pd.read_csv('Obesity Classification.csv')
df.head()

,ID,Age,Gender,Height,Weight,BMI,Label
0,1,25,Male,175,80,25.3,Normal Weight
1,2,30,Female,160,60,22.5,Normal Weight
2,3,35,Male,180,90,27.3,Overweight
3,4,40,Female,150,50,20.0,Underweight
4,5,45,Male,190,100,31.2,Obese


In [3]:
df.Label.unique()

array(['Normal Weight', 'Overweight', 'Underweight', 'Obese'],
      dtype=object)

In [4]:
df.drop(['ID'],axis=1,inplace=True)

In [9]:
df.Label.replace({'Normal Weight':1,'Overweight':2,'Underweight':3,'Obese':4},inplace=True)

/tmp/ipykernel_11035/3918291119.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.Label.replace({'Normal Weight':1,'Overweight':2,'Underweight':3,'Obese':4},inplace=True)


In [10]:
df.head()

,Age,Gender,Height,Weight,BMI,Label
0,25,Male,175,80,25.3,1
1,30,Female,160,60,22.5,1
2,35,Male,180,90,27.3,2
3,40,Female,150,50,20.0,3
4,45,Male,190,100,31.2,4


In [29]:
df.Gender.replace({'Male':1,'Female':2},inplace=True)

/tmp/ipykernel_11035/1830982135.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.Gender.replace({'Male':1,'Female':2},inplace=True)


In [11]:
df.dropna(axis=1,inplace=True)

In [14]:
df.to_csv('obese_new.csv',index=False)

In [15]:
import sagemaker
sess = sagemaker.Session()
uri = sess.upload_data(path='obese_new.csv')
print(uri)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-179201351861/data/obese_new.csv


In [16]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()

In [30]:
X = df.drop(['Label'],axis=1)
y = df['Label']

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)
train_df.to_csv('train_data.csv', index=False, header=0)
test_df.to_csv('test_data.csv', index=False, header=0)

In [32]:
X_train.to_csv('X_train.csv', index=False, header=0)
X_test.to_csv('X_test.csv', index=False, header=0)
y_train.to_csv('y_train.csv', index=False, header=0)
y_test.to_csv('y_test.csv', index=False, header=0)


In [33]:
bucket = sagemaker.Session().default_bucket()
boto3.Session().resource('s3').Bucket(bucket).Object('train_data.csv').upload_file('train_data.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('test_data.csv').upload_file('test_data.csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [64]:
!python train.py --max_leaf_nodes 10

In [72]:
import boto3

def upload_file_to_s3(file_path, bucket_name, s3_key):
    s3_client = boto3.client('s3')
    s3_client.upload_file(file_path, bucket_name, s3_key)

local_file_path = 'model/model.joblib'
bucket_name = 'sagemaker-us-east-1-179201351861'
s3_key = 'model.joblib'

# Upload the file to S3
upload_file_to_s3(local_file_path, bucket_name, s3_key)


In [85]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel

# Define the S3 location of the model data
model_data = 's3://sagemaker-us-east-1-179201351861/model.joblib'

# Define the IAM role ARN
role = 'arn:aws:iam::179201351861:role/service-role/AmazonSageMaker-ExecutionRole-20240330T204113'

entry_point = 'train.py'

sklearn_model = SKLearnModel(model_data=model_data,
                             role=role,
                             entry_point=entry_point,
                             framework_version='0.23-1',  
                             py_version='py3')

# predictor = sklearn_model.deploy(instance_type='ml.m5.large',initial_instance_count =1)
sklearn_model